## OpenAI Assistants in AutoGen

This notebook shows a very basic example of the [`GPTAssistantAgent`](https://github.com/microsoft/autogen/blob/main/autogen/agentchat/contrib/gpt_assistant_agent.py#L16C43-L16C43), which is an experimental AutoGen agent class that leverages the [OpenAI Assistant API](https://platform.openai.com/docs/assistants/overview) for conversational capabilities,  working with
`UserProxyAgent` in AutoGen.

In [1]:
import logging
import os
 
from autogen import config_list_from_json
from autogen import AssistantAgent
from autogen.agentchat.contrib.gpt_assistant_agent import GPTAssistantAgent
from autogen import UserProxyAgent

logger = logging.getLogger(__name__)
logger.setLevel(logging.WARNING)

assistant_id = os.environ.get("ASSISTANT_ID", None)

config_list = config_list_from_json("OAI_CONFIG_LIST")
llm_config = {
    "config_list": config_list,
    "assistant_id": assistant_id
}

gpt_assistant = GPTAssistantAgent(name="assistant",
                                instructions=AssistantAgent.DEFAULT_SYSTEM_MESSAGE,
                                llm_config=llm_config)

user_proxy = UserProxyAgent(name="user_proxy",
    code_execution_config={
        "work_dir": "coding"
    },
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1)
user_proxy.initiate_chat(gpt_assistant, message="Print hello world")

GPT Assistant only supports one OpenAI client. Using the first client in the list.
assistant assistant does not exist, creating a new assistant


user_proxy (to assistant):

Print hello world

--------------------------------------------------------------------------------
assistant (to user_proxy):

```python
# filename: hello_world.py
print("Hello, world!")
```

Please save the above code into a file named `hello_world.py` and then execute it with Python to print "Hello, world!".


--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...


execute_code was called without specifying a value for use_docker. Since the python docker package is not available, code will be run natively. Note: this fallback behavior is subject to change
SIGALRM is not supported on Windows. No timeout will be enforced.


user_proxy (to assistant):

exitcode: 0 (execution succeeded)
Code output: 
Hello, world!


--------------------------------------------------------------------------------
assistant (to user_proxy):

The Python script executed successfully and printed "Hello, world!" as intended.

TERMINATE


--------------------------------------------------------------------------------


In [2]:
user_proxy.initiate_chat(gpt_assistant, message="Write py code to eval 2 + 2", clear_history=True)

user_proxy (to assistant):

Write py code to eval 2 + 2

--------------------------------------------------------------------------------
assistant (to user_proxy):

```python
# This Python code will evaluate the expression 2 + 2 and print the result

# Evaluate the expression
result = 2 + 2

# Print the result
print("The result of 2 + 2 is:", result)
```

You can run this Python code in your environment, and it will output the result of the expression 2 + 2.


--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...


execute_code was called without specifying a value for use_docker. Since the python docker package is not available, code will be run natively. Note: this fallback behavior is subject to change
SIGALRM is not supported on Windows. No timeout will be enforced.


user_proxy (to assistant):

exitcode: 0 (execution succeeded)
Code output: 
The result of 2 + 2 is: 4


--------------------------------------------------------------------------------
assistant (to user_proxy):

The execution of the code was successful, and the result of the expression 2 + 2 is 4.

If you need any more assistance or have any other tasks, feel free to let me know.

TERMINATE


--------------------------------------------------------------------------------


### Teachable GPTAssistantAgent

We can make any conversable agent teachable by adding a Teachability object to it.

In [3]:
from autogen.agentchat.contrib.capabilities.teachability import Teachability
teachability = Teachability(reset_db=True, llm_config={"config_list": config_list})
teachability.add_to_agent(gpt_assistant)


CLEARING MEMORY


For the following tests, we limit each chat to a single conversation turn.

In [4]:
user_proxy.update_max_consecutive_auto_reply(0)

Now let's teach the assistant something it wouldn't know.

In [5]:
user_proxy.initiate_chat(gpt_assistant, message="My favorite color is teal.", clear_history=True)

user_proxy (to assistant):

My favorite color is teal.

--------------------------------------------------------------------------------
assistant (to user_proxy):

Understood, your favorite color is teal. I will remember that for future interactions. Is there anything else you would like to tell me or any task you need assistance with?


--------------------------------------------------------------------------------


Finally, let's start a new conversation and see whether `gpt_assistant` remembers what we taught it in the previous chat.

In [6]:
user_proxy.initiate_chat(gpt_assistant, message="What's my favorite color?", clear_history=True)

user_proxy (to assistant):

What's my favorite color?

--------------------------------------------------------------------------------
assistant (to user_proxy):

Your favorite color is teal. 

TERMINATE


--------------------------------------------------------------------------------


In [7]:
gpt_assistant.delete_assistant()

Permanently deleting assistant...
